In [18]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
import csv

class mevzuat:
    def __init__(self):
        self.new_file()
        self.header = ["kategori", "baslik", "kanun_no", "mevzuat_tarihi", "rega_tarihi ve rega_no", "data_text"]
        with open('C:/Users/omerb/OneDrive/Masaüstü/teknoconcon/Lexlata/Mevzuat Scrape/mevzuat.csv', 'w', encoding='UTF8', newline='') as self.file:
            self.writer = csv.writer(self.file)
            self.writer.writerow(self.header)
            self.i = 1
            while self.i <= 10:
                self.open_homepage()
                self.select_legislation(self.i)
                self.change_frame()
                self.write_data_to_csv()
                self.i += 1
        
    def open_homepage(self):
        # options to open driver
        options = Options()
        options.add_argument('start-maximized')
        options.add_argument('disable-infobars')
        options.add_argument('--disable-extensions')

        # create web driver element
        self.driver = webdriver.Chrome(chrome_options=options, executable_path="C:/Users/omerb/OneDrive/Masaüstü/selenium/chromedriver.exe")
        URL = 'https://www.mevzuat.gov.tr/#kanunlar'
        self.driver.get(URL)

        # click on 'ARA' button
        element = self.driver.find_element(By.ID, 'btnSearch')
        action = ActionChains(self.driver)
        action.click(on_element=element)
        action.perform()

        # open website
        sleep(4)
    
    def select_legislation(self, i):
        content_xpath = f'//*[@id="DataTables_Table_0"]/tbody/tr[{i}]/td[2]/a'
        link = self.driver.find_element(By.XPATH, content_xpath)
        href = link.get_attribute('href')
        self.driver.get(href)
        
    def change_frame(self):
        iframe = self.driver.find_element(By.TAG_NAME, 'iframe')
        self.driver.switch_to.frame(iframe)
        
    def new_file(self):
        newfile = open('C:/Users/omerb/OneDrive/Masaüstü/teknoconcon/Lexlata/Mevzuat Scrape/mevzuat.csv', 'w')
        newfile.close()
        
    def write_data_to_csv(self):   
        text = self.driver.find_elements_by_class_name('MsoNormal')
        toplam = ''
        data = ["kanun"]
         # This for loop gets data that matchs with header array's topics. Except content (icerik).
        for ad in text[:5]:
            if ad.text == ' ':
                continue
            else:
                data.append(ad.text)
                print(ad.text)

        self.writer.writerow(data)

        data = []
    # This for loop gets content (icerik) of legislation. 
        for zort in text[4:]:
            if zort.text == ' ':
                continue
            else:
                #print(zort.text)
                toplam = toplam + zort.text
        data.append(str(toplam))
        self.writer.writerow(data)

            
mevzuat()

C:\Users\omerb\anaconda3\envs\chatbot\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


DEVLET MEMURLARI KANUNU İLE BAZI KANUNLARDA VE 375 SAYILI KANUN HÜKMÜNDE KARARNAMEDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN
Kanun Numarası                    : 7417
Kabul Tarihi                           : 1/7/2022
Yayımlandığı Resmî Gazete   : Tarih: 5/7/2022        Sayı: 31887
İSTANBUL FİNANS MERKEZİ KANUNU
Kanun Numarası                    : 7412
Kabul Tarihi                           : 22/6/2022
Yayımlandığı Resmî Gazete   : Tarih: 28/6/2022          Sayı: 31880
SPOR KULÜPLERİ VE SPOR FEDERASYONLARI KANUNU
Kanun Numarası                    : 7405
Kabul Tarihi                            : 22/4/2022
Yayımlandığı Resmî Gazete   : Tarih: 26/4/2022         Sayı: 31821
NÜKLEER DÜZENLEME KANUNU
Kanun Numarası                    : 7381
Kabul Tarihi                            : 5/3/2022
Yayımlandığı Resmî Gazete   : Tarih: 8/3/2022         Sayı: 31772
ÖĞRETMENLİK MESLEK KANUNU
Kanun Numarası                    : 7354
Kabul Tarihi                            : 3/2/2022
Yayımlandığı Resmî Gazete   